In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Mount Google Drive to access your dataset if needed
from google.colab import drive
drive.mount('/content/drive')

# Define dataset paths
dataset_path = '/content/drive/MyDrive/Eye_diseases'

# Step 1: Load Datasets using image_dataset_from_directory
batch_size = 16
img_size = (224, 224)

# Load the full dataset
full_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    seed=42,
)

# Retrieve class names
class_names = full_dataset.class_names
print(f"Class names: {class_names}")

# Step 2: Split the dataset into training, validation, and test sets
train_size = 0.6
val_size = 0.2
test_size = 0.2

# Calculate sizes
total_size = len(full_dataset)
train_size = int(total_size * train_size)
val_size = int(total_size * val_size)
test_size = total_size - train_size - val_size

# Shuffle and split the dataset
full_dataset = full_dataset.shuffle(buffer_size=1000, seed=42)
train_dataset = full_dataset.take(train_size)
remaining_dataset = full_dataset.skip(train_size)
val_dataset = remaining_dataset.take(val_size)
test_dataset = remaining_dataset.skip(val_size)

# Step 3: Normalize the datasets
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_dataset = train_dataset.map(normalize_img)
val_dataset = val_dataset.map(normalize_img)
test_dataset = test_dataset.map(normalize_img)

# Step 4: Load Pre-Trained Model (MobileNetV2)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)  # Reduced size for small dataset
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(len(class_names), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Step 5: Train the Model
epochs = 32  # Adjusted epochs for smaller dataset

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    verbose=1,
)

# Step 6: Evaluate the Model
loss, accuracy = model.evaluate(test_dataset, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Step 7: Save the Model
# Save the model in the SavedModel format
saved_model_path = "/content/drive/MyDrive/eye_disease_saved_model.h5"
model.save(saved_model_path)
print(f"Model saved in SavedModel format at {saved_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 383 files belonging to 5 classes.
Class names: ['Bulging_Eyes', 'Cataracts', 'Crossed_Eyes', 'Glaucoma', 'Uveitis']
Epoch 1/32
14/14 ━━━━━━━━━━━━━━━━━━━━ 17s 540ms/step - accuracy: 0.3791 - loss: 1.8378 - val_accuracy: 0.5714 - val_loss: 1.0996
Epoch 2/32
14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - accuracy: 0.6108 - loss: 1.1308 - val_accuracy: 0.5938 - val_loss: 0.9863
Epoch 3/32
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 219ms/step - accuracy: 0.6261 - loss: 0.9924 - val_accuracy: 0.5781 - val_loss: 0.9122
Epoch 4/32
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 337ms/step - accuracy: 0.7039 - loss: 0.7866 - val_accuracy: 0.8125 - val_loss: 0.5539
Epoch 5/32
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.6813 - loss: 0.7638 - val_accuracy: 0.8906 - val_loss: 0.3910
Epoch 6/32
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 132ms/step - accuracy: 0.7670 - loss: 0.6527 - val_accuracy: 0.875

Test Accuracy: 100.00%
Model saved in SavedModel format at /content/drive/MyDrive/eye_disease_saved_model.h5
